In [39]:
import re, requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from gamelogs_to_mongo import format_injury_df
from pymongo import MongoClient
from data_cleaning import *
import matplotlib.pyplot as plt

import string
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.util import ngrams

In [13]:
stopwords_ = "a,able,about,across,after,all,almost,also,am,among,an,and,any,\
are,as,at,be,because,been,but,by,can,could,dear,did,do,does,either,\
else,ever,every,for,from,get,got,had,has,have,he,her,hers,him,his,\
how,however,i,if,in,into,is,it,its,just,least,let,like,likely,may,\
me,might,most,must,my,neither,no,of,off,often,on,only,or,other,our,\
own,rather,said,say,says,she,should,since,so,some,than,that,the,their,\
them,then,there,these,they,this,tis,to,too,twas,us,wants,was,we,were,\
what,when,where,which,while,who,whom,why,will,with,would,yet,you,your".split(',')

In [2]:
df0 = pd.read_pickle('./df.pkl')
players = df0['bbref_id'].unique()
df_raw = format_injury_df(df0)

In [133]:
df = df_raw.copy()
df

,Date,player,Team,Status,Notes,bbref_id,from,to,pos,height,...,age,Season,Return_Date,Inj_Duration,New_Inj,Out_of_NBA,Career,League_Years,Num_Inj_Career,Num_Inj_Season
356,1980-10-09,Tree Rollins,Hawks,Injured,placed on IL,rollitr01,1978,1995,C,85,...,25.32,1981,1980-10-24,15 days,1,0,0,3,1,1
368,1980-11-07,Tree Rollins,Hawks,Injured,placed on IL,rollitr01,1978,1995,C,85,...,25.40,1981,1980-11-18,11 days,1,0,0,3,2,2
370,1980-11-17,Rick Mahorn,Bullets,Injured,sprained ankle (DTD),mahorri01,1981,1999,C-F,82,...,22.16,1981,1980-11-21,4 days,1,0,0,0,1,1
377,1980-12-05,John Long,Pistons,Injured,placed on IL,longjo01,1979,1997,G-F,77,...,24.27,1981,1980-12-17,12 days,1,0,0,2,1,1
387,1980-12-21,John Long,Pistons,Injured,placed on IL with knee injury,longjo01,1979,1997,G-F,77,...,24.32,1981,1981-01-02,12 days,1,0,0,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62219,2021-06-07,Jordan Nwora,Bucks,Injured,placed on IL with left thigh injury,nworajo01,2021,2021,F,80,...,22.74,2021,2021-06-25,18 days,1,0,0,0,2,2
62221,2021-06-08,Mike Conley,Jazz,Injured,placed on IL with strained right hamstring,conlemi01,2008,2021,G,73,...,33.66,2021,2021-06-18,10 days,1,0,0,13,46,9
62223,2021-06-09,Sam Merrill,Bucks,Injured,placed on IL with sprained right ankle,merrisa01,2021,2021,G,76,...,25.07,2021,2021-06-13,4 days,1,0,0,0,2,2
62237,2021-06-16,Chris Paul,Suns,Injured,placed on IL with COVID-19 protocols,paulch01,2006,2021,G,72,...,36.11,2021,2021-06-24,8 days,1,0,0,15,43,2


In [131]:
df_new = df[df.New_Inj == True].copy()
df_new

,Date,player,Team,Status,Notes,bbref_id,from,to,pos,height,...,age,Season,Return_Date,Inj_Duration,New_Inj,Out_of_NBA,Career,League_Years,Num_Inj_Career,Num_Inj_Season
356,1980-10-09,Tree Rollins,Hawks,Injured,placed on IL,rollitr01,1978,1995,C,85,...,25.32,1981,1980-10-24,15 days,1,0,0,3,1,1
368,1980-11-07,Tree Rollins,Hawks,Injured,placed on IL,rollitr01,1978,1995,C,85,...,25.40,1981,1980-11-18,11 days,1,0,0,3,2,2
370,1980-11-17,Rick Mahorn,Bullets,Injured,sprained ankle (DTD),mahorri01,1981,1999,C-F,82,...,22.16,1981,1980-11-21,4 days,1,0,0,0,1,1
377,1980-12-05,John Long,Pistons,Injured,placed on IL,longjo01,1979,1997,G-F,77,...,24.27,1981,1980-12-17,12 days,1,0,0,2,1,1
387,1980-12-21,John Long,Pistons,Injured,placed on IL with knee injury,longjo01,1979,1997,G-F,77,...,24.32,1981,1981-01-02,12 days,1,0,0,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62219,2021-06-07,Jordan Nwora,Bucks,Injured,placed on IL with left thigh injury,nworajo01,2021,2021,F,80,...,22.74,2021,2021-06-25,18 days,1,0,0,0,2,2
62221,2021-06-08,Mike Conley,Jazz,Injured,placed on IL with strained right hamstring,conlemi01,2008,2021,G,73,...,33.66,2021,2021-06-18,10 days,1,0,0,13,46,9
62223,2021-06-09,Sam Merrill,Bucks,Injured,placed on IL with sprained right ankle,merrisa01,2021,2021,G,76,...,25.07,2021,2021-06-13,4 days,1,0,0,0,2,2
62237,2021-06-16,Chris Paul,Suns,Injured,placed on IL with COVID-19 protocols,paulch01,2006,2021,G,72,...,36.11,2021,2021-06-24,8 days,1,0,0,15,43,2


In [143]:
notes = df.Notes
notes_list = notes.tolist()
notes_new = df_new.Notes


In [144]:
tokens = notes.apply(lambda x: word_tokenize(x.lower()))
tokens_new = notes_new.apply(lambda x: word_tokenize(x.lower()))
tokens

356                                       [placed, on, il]
368                                       [placed, on, il]
370                           [sprained, ankle, (, dtd, )]
377                                       [placed, on, il]
387                   [placed, on, il, with, knee, injury]
                               ...                        
62219          [placed, on, il, with, left, thigh, injury]
62221    [placed, on, il, with, strained, right, hamstr...
62223       [placed, on, il, with, sprained, right, ankle]
62237          [placed, on, il, with, covid-19, protocols]
62240                  [sprained, right, ankle, (, dtd, )]
Name: Notes, Length: 43099, dtype: object

In [145]:
punctuation_ = set(string.punctuation)
def filter_tokens(sent):
    return([w for w in sent if not w in stopwords_ and not w in punctuation_])
tokens_filtered = tokens.apply(lambda x: filter_tokens(x))
tokens_filtered_new = tokens_new.apply(lambda x: filter_tokens(x))
tokens_filtered

356                                  [placed, il]
368                                  [placed, il]
370                        [sprained, ankle, dtd]
377                                  [placed, il]
387                    [placed, il, knee, injury]
                           ...                   
62219           [placed, il, left, thigh, injury]
62221    [placed, il, strained, right, hamstring]
62223        [placed, il, sprained, right, ankle]
62237           [placed, il, covid-19, protocols]
62240               [sprained, right, ankle, dtd]
Name: Notes, Length: 43099, dtype: object

In [146]:
stemmer_porter, stemmer_snowball, lemmatizer = PorterStemmer(), SnowballStemmer('english'),WordNetLemmatizer()
tokens_stemporter = tokens_filtered.apply(lambda x: [stemmer_porter.stem(w) for w in x])
tokens_stemsnowball = tokens_filtered.apply(lambda x: [stemmer_snowball.stem(w) for w in x])
tokens_lemmatizer = tokens_filtered.apply(lambda x: [lemmatizer.lemmatize(w) for w in x])
tokens_stemporter_new = tokens_filtered_new.apply(lambda x: [stemmer_porter.stem(w) for w in x])
tokens_stemsnowball_new = tokens_filtered_new.apply(lambda x: [stemmer_snowball.stem(w) for w in x])
tokens_lemmatizer_new = tokens_filtered_new.apply(lambda x: [lemmatizer.lemmatize(w) for w in x])

In [150]:
stem_lemm_df = pd.DataFrame({'porter':tokens_stemporter, 'snowball':tokens_stemsnowball, 'lemmatizer': tokens_lemmatizer})
stem_lemm_df_new = pd.DataFrame({'porter_new':tokens_stemporter_new, 'snowball_new':tokens_stemsnowball_new, 'lemmatizer_new': tokens_lemmatizer_new})


stem_lemm_df
stem_lemm_df_new

,porter_new,snowball_new,lemmatizer_new
356,"[place, il]","[place, il]","[placed, il]"
368,"[place, il]","[place, il]","[placed, il]"
370,"[sprain, ankl, dtd]","[sprain, ankl, dtd]","[sprained, ankle, dtd]"
377,"[place, il]","[place, il]","[placed, il]"
387,"[place, il, knee, injuri]","[place, il, knee, injuri]","[placed, il, knee, injury]"
...,...,...,...
62219,"[place, il, left, thigh, injuri]","[place, il, left, thigh, injuri]","[placed, il, left, thigh, injury]"
62221,"[place, il, strain, right, hamstr]","[place, il, strain, right, hamstr]","[placed, il, strained, right, hamstring]"
62223,"[place, il, sprain, right, ankl]","[place, il, sprain, right, ankl]","[placed, il, sprained, right, ankle]"
62237,"[place, il, covid-19, protocol]","[place, il, covid-19, protocol]","[placed, il, covid-19, protocol]"


In [151]:
stem_lemm_df[tokens_stemporter != tokens_stemsnowball]
stem_lemm_df_new[tokens_stemporter_new != tokens_stemsnowball_new]

,porter_new,snowball_new,lemmatizer_new
1160,"[stomach, viru, dtd]","[stomach, virus, dtd]","[stomach, virus, dtd]"
1359,"[small, tear, later, meniscu, right, knee, dtd]","[small, tear, later, meniscus, right, knee, dtd]","[small, tear, lateral, meniscus, right, knee, ..."
1443,"[ruptur, right, achil, tendon, out, season]","[ruptur, right, achill, tendon, out, season]","[ruptured, right, achilles, tendon, out, season]"
1604,"[injuri, sinu, caviti, out, 10, day]","[injuri, sinus, caviti, out, 10, day]","[injury, sinus, cavity, out, 10, day]"
1611,"[torn, achil, tendon]","[torn, achill, tendon]","[torn, achilles, tendon]"
...,...,...,...
62057,"[place, il, right, achil, injuri]","[place, il, right, achill, injuri]","[placed, il, right, achilles, injury]"
62061,"[right, achil, injuri, dtd]","[right, achill, injuri, dtd]","[right, achilles, injury, dtd]"
62084,"[place, il, left, achil, injuri]","[place, il, left, achill, injuri]","[placed, il, left, achilles, injury]"
62129,"[place, il, left, achil, injuri]","[place, il, left, achill, injuri]","[placed, il, left, achilles, injury]"


In [157]:
for token in [tokens_stemporter, tokens_stemsnowball, tokens_lemmatizer]:
    term_occurance = Counter(x for row in token for x in row)
    print(term_occurance.most_common(75),'\n')

[('dnp', 18429), ('place', 16877), ('il', 16115), ('left', 12825), ('right', 12810), ('knee', 7278), ('sprain', 6715), ('ankl', 5885), ('sore', 4904), ('strain', 4834), ('dtd', 4800), ('injuri', 3702), ('back', 3168), ('out', 3074), ('surgeri', 2661), ('bruis', 2329), ('foot', 2224), ('lower', 1520), ('hamstr', 1509), ('season', 1494), ('fractur', 1437), ('torn', 1342), ('tendin', 1299), ('shoulder', 1295), ('groin', 1212), ('recov', 1124), ('spasm', 1034), ('repair', 1015), ('calf', 983), ('hip', 970), ('indefinit', 909), ('flu', 907), ('ill', 816), ('achil', 795), ('rest', 776), ('wrist', 768), ('tendon', 763), ('ir', 762), ('bone', 722), ('toe', 567), ('thumb', 564), ('hand', 545), ('week', 544), ('broken', 524), ('quadricep', 515), ('elbow', 481), ('finger', 480), ('stress', 458), ('concuss', 455), ('leg', 427), ('arthroscop', 398), ('plantar', 364), ('muscl', 363), ('abdomin', 341), ('infect', 319), ('stomach', 313), ('thigh', 306), ('big', 305), ('acl', 299), ('ligament', 293), (

In [156]:
for token in [tokens_stemporter_new, tokens_stemsnowball_new, tokens_lemmatizer_new]:
    term_occurance = Counter(x for row in token for x in row)
    print(term_occurance.most_common(75),'\n')


[('place', 11896), ('il', 11436), ('dnp', 7102), ('left', 6508), ('right', 6383), ('dtd', 4145), ('knee', 3711), ('sprain', 3517), ('ankl', 3177), ('sore', 2865), ('injuri', 2512), ('strain', 2300), ('back', 1896), ('out', 1885), ('bruis', 1337), ('foot', 987), ('lower', 845), ('tendin', 829), ('season', 825), ('hamstr', 757), ('rest', 727), ('shoulder', 712), ('surgeri', 703), ('flu', 697), ('spasm', 680), ('ill', 670), ('indefinit', 625), ('groin', 589), ('hip', 574), ('calf', 528), ('fractur', 504), ('torn', 474), ('ir', 460), ('achil', 423), ('wrist', 394), ('week', 359), ('quadricep', 316), ('toe', 308), ('tendon', 303), ('thumb', 290), ('finger', 276), ('bone', 270), ('hand', 258), ('concuss', 254), ('broken', 252), ('stomach', 245), ('repair', 239), ('elbow', 239), ('leg', 221), ('thigh', 185), ('infect', 183), ('nba', 175), ('protocol', 171), ('health', 168), ('safeti', 168), ('neck', 167), ('muscl', 161), ('arthroscop', 159), ('viru', 156), ('patella', 156), ('plantar', 154), 